In [28]:
# 기본
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
from matplotlib import font_manager, rc
import platform
from tqdm import tqdm
import sklearn
from sklearn import linear_model
import scipy.stats as stats
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
from patsy import dmatrices
from sklearn.model_selection import train_test_split

# time
import datetime

# crawling
import requests
import lxml.html
import sqlite3
from pandas.io import sql
from bs4 import BeautifulSoup

# 한글 폰트 설정
if platform.system() == 'Windows':
# 윈도우인 경우
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)

In [ ]:
ukb37332 = pd.read_sas('SAS\\UKBIOBANK\\ukb37332.sas7bdat', format = 'sas7bdat')
ukb37332

In [ ]:
ukb45783_data = []
ukb45783 = pd.read_sas('SAS\\UKBIOBANK\\ukb45783.sas7bdat', format = 'sas7bdat', chunksize=100000, iterator=True)

for chunk in ukb45783:
    ukb45783_data.append(chunk)
asm_data_df=pd.concat(ukb45783_data)

In [ ]:
ukb46987 = pd.read_sas('SAS\\UKBIOBANK\\ukb46987.sas7bdat', format = 'sas7bdat')
ukb46987

In [ ]:
ukb46987 = pd.read_sas('SAS\\ukb46987_new.sas7bdat', format = 'sas7bdat', encoding = 'euc-kr')
ukb46987

In [ ]:
df = ukb46987.copy()

In [ ]:
df2 = df.iloc[:, :224]
df2

In [ ]:
df4 = df.iloc[:, 224:]
df4

In [ ]:
# colnames = df2.columns.tolist()
# colnames.remove('eid')

# n18 = 0
# for row in range(len(df2)): 
#     for col in colnames: 
#         print(df2[col][row])
#         if df2[col][row] is not np.nan: 
#             disease = df2[col][row].decode('euc-kr')
#             if disease.startswith('N18') == True: 
#                 n18 += 1
#                 break

In [ ]:
df2['N180'] = 0
df2['N181'] = 0
df2['N182'] = 0
df2['N183'] = 0
df2['N184'] = 0
df2['N185'] = 0
df2['N186'] = 0
df2['N187'] = 0
df2['N188'] = 0
df2['N189'] = 0

In [ ]:
df2

In [ ]:
colnames = df2.columns.tolist()
colnames.remove('eid')

for col in colnames: 
    df2.loc[df2[col] == 'N180', 'N180'] = 1  # 전체 222개 칼럼 순회하면서 N180 있으면 1 넣어주기
    df2.loc[df2[col] == 'N180', 'N180_index'] = col[6:]  # 41280 변수에서 N18 진단받은 위치와 동일한 위치에 있는 날짜 뽑아오기 위해 칼럼명을 잘라서 인덱스만 가져오기

    #df2["N180"] = df2.apply(lambda x: 1 if x[col] == 'N180' else 0, axis=0)

In [ ]:
df2['N180'].value_counts() 

In [ ]:
# 인덱스랑 진단여부 갯수 똑같은 거 확인

df2['N180_index'].isna().sum()

In [ ]:
for col in colnames: 
    df2.loc[df2[col] == 'N181', 'N181'] = 1  # 전체 222개 칼럼 순회하면서 N181 있으면 1 넣어주기
    df2.loc[df2[col] == 'N181', 'N181_index'] = col[6:]  # 41280 변수에서 N18 진단받은 위치와 동일한 위치에 있는 날짜 뽑아오기 위해 칼럼명을 잘라서 인덱스만 가져오기

In [ ]:
for col in colnames: 
    df2.loc[df2[col] == 'N182', 'N182'] = 1  # 전체 222개 칼럼 순회하면서 N182 있으면 1 넣어주기
    df2.loc[df2[col] == 'N182', 'N182_index'] = col[6:]  # 41280 변수에서 N18 진단받은 위치와 동일한 위치에 있는 날짜 뽑아오기 위해 칼럼명을 잘라서 인덱스만 가져오기

In [ ]:
for col in colnames: 
    df2.loc[df2[col] == 'N183', 'N183'] = 1  # 전체 222개 칼럼 순회하면서 N183 있으면 1 넣어주기
    df2.loc[df2[col] == 'N183', 'N183_index'] = col[6:]  # 41280 변수에서 N18 진단받은 위치와 동일한 위치에 있는 날짜 뽑아오기 위해 칼럼명을 잘라서 인덱스만 가져오기

In [ ]:
for col in colnames: 
    df2.loc[df2[col] == 'N184', 'N184'] = 1  # 전체 222개 칼럼 순회하면서 N184 있으면 1 넣어주기
    df2.loc[df2[col] == 'N184', 'N184_index'] = col[6:]  # 41280 변수에서 N18 진단받은 위치와 동일한 위치에 있는 날짜 뽑아오기 위해 칼럼명을 잘라서 인덱스만 가져오기

In [ ]:
for col in colnames: 
    df2.loc[df2[col] == 'N185', 'N185'] = 1  # 전체 222개 칼럼 순회하면서 N185 있으면 1 넣어주기
    df2.loc[df2[col] == 'N185', 'N185_index'] = col[6:]  # 41280 변수에서 N18 진단받은 위치와 동일한 위치에 있는 날짜 뽑아오기 위해 칼럼명을 잘라서 인덱스만 가져오기

In [ ]:
for col in colnames: 
    df2.loc[df2[col] == 'N186', 'N186'] = 1  # 전체 222개 칼럼 순회하면서 N186 있으면 1 넣어주기
    df2.loc[df2[col] == 'N186', 'N186_index'] = col[6:]  # 41280 변수에서 N18 진단받은 위치와 동일한 위치에 있는 날짜 뽑아오기 위해 칼럼명을 잘라서 인덱스만 가져오기

In [ ]:
for col in colnames: 
    df2.loc[df2[col] == 'N187', 'N187'] = 1  # 전체 222개 칼럼 순회하면서 N187 있으면 1 넣어주기
    df2.loc[df2[col] == 'N187', 'N187_index'] = col[6:]  # 41280 변수에서 N18 진단받은 위치와 동일한 위치에 있는 날짜 뽑아오기 위해 칼럼명을 잘라서 인덱스만 가져오기

In [ ]:
for col in colnames: 
    df2.loc[df2[col] == 'N188', 'N188'] = 1  # 전체 222개 칼럼 순회하면서 N188 있으면 1 넣어주기
    df2.loc[df2[col] == 'N188', 'N188_index'] = col[6:]  # 41280 변수에서 N18 진단받은 위치와 동일한 위치에 있는 날짜 뽑아오기 위해 칼럼명을 잘라서 인덱스만 가져오기

In [ ]:
for col in colnames: 
    df2.loc[df2[col] == 'N189', 'N189'] = 1  # 전체 222개 칼럼 순회하면서 N189 있으면 1 넣어주기
    df2.loc[df2[col] == 'N189', 'N189_index'] = col[6:]  # 41280 변수에서 N18 진단받은 위치와 동일한 위치에 있는 날짜 뽑아오기 위해 칼럼명을 잘라서 인덱스만 가져오기

In [ ]:
# colnames = df2.columns.tolist()
# colnames.remove('eid')

# for col in colnames: 
#     print(col, ' ', df2.loc[df2[col] == 'N181', col].count())

In [ ]:
# for col in colnames: 
#     df["N180"] = df.apply(lambda x: 1 if x[col] == 'N180' else 0, axis = 1)
#     df["N181"] = df.apply(lambda x: 1 if x[col] == 'N181' else 0, axis = 1)
#     df["N182"] = df.apply(lambda x: 1 if x[col] == 'N182' else 0, axis = 1)
#     df["N183"] = df.apply(lambda x: 1 if x[col] == 'N183' else 0, axis = 1)
#     df["N184"] = df.apply(lambda x: 1 if x[col] == 'N184' else 0, axis = 1)
#     df["N185"] = df.apply(lambda x: 1 if x[col] == 'N185' else 0, axis = 1)
#     df["N186"] = df.apply(lambda x: 1 if x[col] == 'N186' else 0, axis = 1)
#     df["N187"] = df.apply(lambda x: 1 if x[col] == 'N187' else 0, axis = 1)
#     df["N188"] = df.apply(lambda x: 1 if x[col] == 'N188' else 0, axis = 1)
#     df["N189"] = df.apply(lambda x: 1 if x[col] == 'N189' else 0, axis = 1)

In [ ]:
df2["N180"].value_counts()

In [ ]:
df2['N180_index'].isna().sum()

In [ ]:
df2["N181"].value_counts()

In [ ]:
df2['N181_index'].isna().sum()

In [ ]:
df2["N182"].value_counts()

In [ ]:
df2['N182_index'].isna().sum()

In [ ]:
df2["N183"].value_counts()

In [ ]:
df2['N183_index'].isna().sum()

In [ ]:
df2["N184"].value_counts()

In [ ]:
df2['N184_index'].isna().sum()

In [ ]:
df2["N185"].value_counts()

In [ ]:
df2['N185_index'].isna().sum()

In [ ]:
df2["N186"].value_counts()

In [ ]:
df2['N186_index'].isna().sum()

In [ ]:
df2["N187"].value_counts()

In [ ]:
df2['N187_index'].isna().sum()

In [ ]:
df2["N188"].value_counts()

In [ ]:
df2['N188_index'].isna().sum()

In [ ]:
df2["N189"].value_counts()

In [ ]:
df2['N189_index'].isna().sum()

In [ ]:
df_final = df2[['eid', 'N180', 'N181', 'N182', 'N183', 'N184', 'N185', 'N186', 'N187', 'N188', 'N189', 
                'N180_index', 'N181_index', 'N182_index', 'N183_index', 'N184_index', 'N185_index', 'N186_index', 'N187_index', 'N188_index', 'N189_index']]
df_final

In [ ]:
df4

In [ ]:
df5 = pd.concat([df_final, df4], axis = 1)
df5

In [ ]:
dftmp = df5[['N180_index', 'N181_index', 'N182_index', 'N183_index', 'N184_index', 'N185_index', 'N186_index', 'N187_index', 'N188_index', 'N189_index']].copy()

In [ ]:
dftmp['N180_index'].unique()

In [ ]:
#### TEST

coltest = ['idx1', 'idx2', 'idx3', '41280-1.0', '41280-5.0', '41280-9.0']
value = [[1, np.nan, np.nan, '2010-01-01', np.nan, np.nan], [np.nan, 5, np.nan, np.nan, '2010-03-03', np.nan], [np.nan, np.nan, 9, np.nan, np.nan, '2021-01-01'], [np.nan, np.nan, np.nan, np.nan, np.nan, np.nan]]
testdf = pd.DataFrame(columns = coltest, data=value)

# vltest = testdf[col].sum(axis = 1).tolist()
# vltestdf = pd.DataFrame(vltest)
# vltestdf[0].unique()

testdf['N18_idx'] = testdf.stack().groupby(level=0).first().reindex(testdf.index)   # 각 행에서 Nan 값이 아닌 유일한 값 가져오기
testdf
# testdf['N18_Date'] = list(map(lambda y: np.nan if y is np.nan else testdf['41280-' + str(y)].values[0], testdf['N18_idx']))

In [ ]:
testdf['eid'] = ['A', 'B', 'C', 'D']
valuelist = testdf['N18_idx'].tolist()

testdf['N18_Date'] = list(map(lambda x, y: np.nan if y is np.nan else testdf.loc[testdf['eid'] == x, '41280-' + str(y)].values[0], testdf['eid'], valuelist))

In [ ]:
testdf

In [ ]:
########## test end

In [ ]:
column_list = ['N180_index', 'N181_index', 'N182_index', 'N183_index', 'N184_index', 'N185_index', 'N186_index', 'N187_index', 'N188_index', 'N189_index']

dftmp['N18_idx'] = dftmp.stack().groupby(level=0).first().reindex(dftmp.index)  # 각 행에서 Nan 값이 아닌 유일한 값 가져오기

In [ ]:
dftmp['N18_idx'].unique()

In [ ]:
dftmp

In [ ]:
df5 = pd.concat([df5, dftmp['N18_idx']], axis = 1)
df5

In [ ]:
c = df5.columns.tolist()
for col in c: 
    print(col)

In [ ]:
# 열별로 진단받은 날짜 추가하기

valuelist = df5['N18_idx'].tolist()
df5['N18_Date'] = list(map(lambda x, y: np.nan if y is np.nan else df5.loc[df5['eid'] == x, '41280-' + str(y)].values[0], df5['eid'], valuelist))

In [ ]:
len(df5['N18_Date'].unique())

In [ ]:
df5

In [ ]:
df6 = df5[['eid', 
'N180', 'N181', 'N182', 'N183', 'N184', 'N185', 'N186', 'N187', 'N188', 'N189', 
'N180_index', 'N181_index', 'N182_index', 'N183_index', 'N184_index', 'N185_index', 'N186_index', 'N187_index', 'N188_index', 'N189_index',
'N18_idx', 'N18_Date']]

In [ ]:
df6

In [ ]:
df6.to_csv('data\\ukb46987_N18preprocessed.csv', encoding = 'euc-kr', index = False)

#### 2021.12.06 ~

In [ ]:
df = pd.read_csv('data\\ukb46987_N18preprocessed.csv', encoding = 'euc-kr')
df

In [ ]:
def conditions(df):
    if (df['N180'] == 1) or (df['N181'] == 1) or (df['N182'] == 1) or (df['N183'] == 1) or (df['N184'] == 1) or (df['N185'] == 1) or (df['N186'] == 1) or (df['N187'] == 1) or (df['N188'] == 1) or (df['N189'] == 1):
        return 1
    else:
        return 0

In [ ]:
df['N18'] = df.apply(conditions, axis=1)

In [ ]:
df['N18'].value_counts()

In [ ]:
df['N18_Date'].fillna(0, inplace = True)

In [ ]:
df.loc[df['N18_Date'] == 0, 'N18_Date'].count()

In [ ]:
df

In [ ]:
df_final = df[['eid', 'N18_Date', 'N18']]
df_final

In [ ]:
df_final.to_csv('data\\ukb46987_N18preprocessed.csv', encoding = 'euc-kr', index = False)

#### HTN drug preprocessing

In [ ]:
ukb45783_drug = pd.read_sas('SAS\\ukb45783_drug.sas7bdat', format = 'sas7bdat')
ukb45783_drug

In [ ]:
df = ukb45783_drug.copy()

In [ ]:
HTN_drugcode_preprocessed_FINAL = pd.read_csv('data\\HTN_drugcode_preprocessed_FINAL.csv', encoding = 'euc-kr')
HTN_drugcode_preprocessed_FINAL

In [ ]:
htn_drugcode = HTN_drugcode_preprocessed_FINAL['UK Biobank code'].dropna()
htn_drugcode = htn_drugcode.tolist()
htn_drugcode

In [ ]:
colnames = df.columns.tolist()
colnames.remove('eid')

In [ ]:
for col in colnames: 
    for val in htn_drugcode: 
        df.loc[df[col] == val, str(val)] = 1  # 전체 칼럼 순회하면서 drug code 있으면 칼럼 만들어서 1 넣어주기

In [ ]:
df

In [ ]:
# 엑셀로 정리해둔 명수랑 얼추 맞음

for val in htn_drugcode: 
    print(df[str(val)].value_counts())

In [ ]:
htn_drugcode_str = list(map(str, htn_drugcode))
htn_drugcode_str.insert(0, 'eid')
htn_drugcode_str

In [ ]:
df_final = df[htn_drugcode_str].copy()
df_final

In [ ]:
df_final.fillna(0, inplace = True)
df_final

In [ ]:
#### test

d = {'A':[1, 0, 0, 0, 0], 
          'B':[0, 1, 0, 0, 1], 
          'C':[1, 1, 1, 0, 0], 
          'D':[0, 0, 0, 0, 1], 
          'E':[0, 0, 0, 0, 0], 
          'col':[1, 1, 1, 0, 1]}
testdf = pd.DataFrame(data = d, index = ['prsn1', 'prsn2', 'prsn3', 'prsn4', 'prsn5'])
testdf

In [ ]:
def create_htnmed(df):
    count = 0
    col = pd.DataFrame(df).columns.tolist()
    print(col)

    for c in col: 
        if df[str(c)] == 1: 
            count += 1
    if count > 0: 
        return 1
    else: 
        return 0

In [ ]:
for val in htn_drugcode: 
    df_final.loc[df_final[str(val)] == 1, 'HTN_med'] = 1

In [ ]:
df_final['HTN_med'].fillna(0, inplace = True)

In [ ]:
df_final['HTN_med'].value_counts()

In [ ]:
df_final

In [ ]:
df_final.to_csv('data\\ukb45783_drugpreprocessed.csv', encoding = 'euc-kr', index = False)

#### 2021.12.08 ~     
#### Last follow up date 생성

In [29]:
date_merged = pd.read_sas('SAS\\ukb_date_merged.sas7bdat', format = 'sas7bdat', encoding = 'cp1252')
date_merged

,eid,23048-0.0,23048-1.0,30502-0.0,30502-1.0,30512-0.0,30512-1.0,30522-0.0,30522-1.0,30601-0.0,...,41281-0.38,41281-0.39,41281-0.40,41281-0.41,41281-0.42,41281-0.43,41281-0.44,41281-0.45,41281-0.46,last_followup_date
0,1000015,NaN,NaN,2014-11-08,NaN,2014-11-08,NaN,2014-11-08,NaN,2016-01-10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000015.0
1,1000027,NaN,NaN,2014-12-04,2015-06-08,2014-12-04,2015-06-08,2014-12-04,2015-06-08,2017-02-04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000027.0
2,1000039,NaN,NaN,2014-12-01,NaN,2014-12-01,NaN,2014-12-01,NaN,2017-01-07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000039.0
3,1000040,NaN,NaN,2015-07-12,NaN,2015-07-12,NaN,2015-07-12,NaN,2017-03-07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000040.0
4,1000053,NaN,NaN,2015-01-31,NaN,2015-01-31,NaN,2015-01-31,NaN,2017-08-07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000053.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502470,6025327,NaN,NaN,2014-11-29,NaN,2014-11-29,NaN,2014-11-29,NaN,2016-05-10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6025327.0
502471,6025339,NaN,NaN,2015-02-07,NaN,2015-02-07,NaN,2015-02-07,NaN,2017-04-06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6025339.0
502472,6025340,NaN,NaN,2015-02-21,NaN,2015-02-21,NaN,2015-02-21,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6025340.0
502473,6025353,NaN,NaN,2015-09-17,NaN,2015-09-17,NaN,2015-09-17,NaN,2016-12-20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6025353.0


In [30]:
df = date_merged.copy()

In [31]:
id_tmp = df[['eid']]

In [32]:
id_tmp

,eid
0,1000015
1,1000027
2,1000039
3,1000040
4,1000053
...,...
502470,6025327
502471,6025339
502472,6025340
502473,6025353


In [33]:
df.drop(['eid', 'eid_num', 'last_followup_date'], axis = 1, inplace = True)
df

,23048-0.0,23048-1.0,30502-0.0,30502-1.0,30512-0.0,30512-1.0,30522-0.0,30522-1.0,30601-0.0,30601-1.0,...,41281-0.37,41281-0.38,41281-0.39,41281-0.40,41281-0.41,41281-0.42,41281-0.43,41281-0.44,41281-0.45,41281-0.46
0,NaN,NaN,2014-11-08,NaN,2014-11-08,NaN,2014-11-08,NaN,2016-01-10,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,2014-12-04,2015-06-08,2014-12-04,2015-06-08,2014-12-04,2015-06-08,2017-02-04,2016-10-20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,2014-12-01,NaN,2014-12-01,NaN,2014-12-01,NaN,2017-01-07,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,2015-07-12,NaN,2015-07-12,NaN,2015-07-12,NaN,2017-03-07,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,2015-01-31,NaN,2015-01-31,NaN,2015-01-31,NaN,2017-08-07,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502470,NaN,NaN,2014-11-29,NaN,2014-11-29,NaN,2014-11-29,NaN,2016-05-10,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502471,NaN,NaN,2015-02-07,NaN,2015-02-07,NaN,2015-02-07,NaN,2017-04-06,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502472,NaN,NaN,2015-02-21,NaN,2015-02-21,NaN,2015-02-21,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502473,NaN,NaN,2015-09-17,NaN,2015-09-17,NaN,2015-09-17,NaN,2016-12-20,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df['30502-0.0'][0]

In [34]:
# Nan이 있으면 max value가 다 nan으로 잡힌다

df.fillna('0', inplace = True)

In [38]:
# datetime 형식으로 전부 바꾸기

df.iloc[:, 0:493] = df.iloc[:, 0:493].apply(pd.to_datetime, errors='coerce')

In [ ]:
# df.replace(0, '0', inplace = True)

In [ ]:
df

In [35]:
# # datetime 형식인 열들 str로 바꾸기 - 안됨

# df.iloc[:, 67:120] = df.iloc[:, 67:120].strftime("%Y-%m-%d")

AttributeError: 'DataFrame' object has no attribute 'strftime'

In [36]:
# # 왜 안바뀌지 ..? 전체 다 해보자  - DATETIME 형식을 astype로 str로 바꿀 순 없는 듯 하다

# all_columns = df.columns.tolist()
# df[all_columns] = df[all_columns].astype(str)

In [40]:
df.replace({pd.NaT: np.NaN}, inplace = True)

In [41]:
df.iloc[:, 67:120]

,53-1.0,53-2.0,53-3.0,20078-0.0,20078-1.0,20078-2.0,20078-3.0,20078-4.0,20400-0.0,22500-0.0,...,40005-14.0,40005-15.0,40005-16.0,42000-0.0,42002-0.0,42004-0.0,42006-0.0,42008-0.0,42010-0.0,42012-0.0
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012-05-20 00:00:00,NaN,2015-08-01 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2013-01-14 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-09-20 00:00:00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,2011-03-10 00:00:00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502470,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502471,NaN,NaN,NaN,2009-09-05 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502472,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502473,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-07-09 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
# 왜 안돼 ....

last_followup_date = np.nanmax(df, axis = 1)

TypeError: '>=' not supported between instances of 'Timestamp' and 'float'

In [ ]:
last_followup_date

In [ ]:
df.info()

In [ ]:
pd.set_option('display.max_column', None)
k = df.iloc[[0]]
k

In [ ]:
np.nanmax(k, axis = 1)

In [ ]:
ary = ['2012-05-31', '2013-03-03', '2016-05-05', np.nan]
ary

In [ ]:
ary

In [ ]:
df['30502-0.0']